<a href="https://colab.research.google.com/github/Entaro-zeratul/COVID_SCRAPING/blob/main/%EC%BD%94%EB%A1%9C%EB%82%98%EC%8A%A4%ED%81%AC%EB%9E%A9%ED%95%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 수집 vs 크롤링

웹사이트 내용을 읽어 오는 것 -> 웹 데이터 수집, 스크래핑
검색엔진이 하이퍼링크를 타고 웹 페이지 내용을 읽는 것 -> 크롤링

**Request** --> 작은 브라우저로 웹사이트를 읽어 오는 목적
**beautifulsoup4** --> html 사용

## 지식 저작권 문제
### 로봇 배제 표준
로봇이 robots.txt 를 보고 이 자료를 긁어도 되는지 안되는지에 대한 내용

https://www.seoul.go.kr/robots.txt 이런식으로 사이트 뒤에 robots.txt를 붙여서 확인한다.
allow 는 긁어와도 되는 것이고 
disallow는 안된다.

### 저작권 정책
사이트에서 저작권을 확인한다.

###**굵은 텍스트** 무리한 네트워크 요청
지나친 네트워크 요청(여러 데이터를 동시에 많이 긁어온다)를 한다면 DDOS(서비스 거부 공격) 로 의심 할 수 있다.

# 데이터 수집 방법
1. 수집하고자 하는 url를 알아온다.
2. 파이썬의 작은 브라우저 request를 이용해 url에 접근한다.
3. response.status_code 각 200 OK라면 정상 응답입니다.
4. request의 response.text만 받아옵니다.
5. html을 받아왔다면 bs(response.test, html.parser)로 해석합니다.
6. soup.select를 통해 원하는 태크에 접근합니다.
7. 목록을 받아옵니다.
8. 목록에서 행을 하나씩 가져옵니다.
9. 행을 모아서 데이터 프레임으로 만듭니다.

## JSON이란

JSON(제이슨[1], JavaScript Object Notation)은 속성-값 쌍( attribute–value pairs and array data types (or any other serializable value)) 또는 "키-값 쌍"으로 이루어진 데이터 오브젝트를 전달하기 위해 인간이 읽을 수 있는 텍스트를 사용하는 개방형 표준 포맷이다. 비동기 브라우저/서버 통신 (AJAX)을 위해, 넓게는 XML(AJAX가 사용)을 대체하는 주요 데이터 포맷이다. 특히, 인터넷에서 자료를 주고 받을 때 그 자료를 표현하는 방법으로 알려져 있다. 자료의 종류에 큰 제한은 없으며, 특히 컴퓨터 프로그램의 변수값을 표현하는 데 적합하다.



# 서울시 코로나 데이터 수집 순서
1. 페이지별로 데이터 수집
2. 전체 페이지 수집
3. pd.concat([전체 데이터 리스트])로 데이터를 통합
4. 데이터 전처리(html 태그 제거)
5. to_csv로 데이터 병합
6. pd.read_csv로 잘 저장 되었는지 확인
7. 수집 분석 끝! 분석 시작


In [ ]:
import pandas as pd
import requests
import tqdm


In [ ]:
url = 'https://news.seoul.go.kr/api/27/getCorona19Route/get_route_ajax.php?draw=1'
# url = f'{url}&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=desc'
url = f"{url}&start=0&length=100"
#"&search%5Bvalue%5D=&search%5Bregex%5D=true&_=1608000305693'
url

'https://news.seoul.go.kr/api/27/getCorona19Route/get_route_ajax.php?draw=1&start=0&length=100'

In [ ]:
response = requests.get(url)
data_json = response.json()
pd.DataFrame(data_json["data"])

,0
0,"<div class='moveText'><table class=""tNewStyle ..."
1,"<div class='moveText'><table class=""tNewStyle ..."
2,"<div class='moveText'><table class=""tNewStyle ..."
3,"<div class='moveText'><table class=""tNewStyle ..."
4,"<div class='moveText'><table class=""tNewStyle ..."
...,...
95,"<div class='moveText'><table class=""tNewStyle ..."
96,"<div class='moveText'><table class=""tNewStyle ..."
97,"<div class='moveText'><table class=""tNewStyle ..."
98,"<div class='moveText'><table class=""tNewStyle ..."


In [ ]:
records_total = data_json['recordsTotal']
records_total

463

In [ ]:
# 여러 페이지를 가져오자
def get_seoul_covid20_100(page_no):
    """
    page_no: 입력값으로 페이지 번호를 입력하면 해당 번호의 데이터를 가져옴
    start_no: 입력받은 페이지 넘버
    """
    start_no = (page_no - 1) * 100
    url = f"https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw={page_no}"
    url = f"{url}&order%5B0%5D%5Bdir%5D=desc&start={start_no}&length=100"
    reponse = requests.get(url)
    data_json = reponse.json()
    return data_json

In [ ]:
get_seoul_covid20_100(-1)

{'data': [], 'draw': -1, 'recordsFiltered': 463, 'recordsTotal': 463}

tqdm을 이용해 for문의 진행도를 알아보자 느린 for문이지만 상태 정도르 알 수 있다.


In [ ]:
import time
from tqdm import trange

In [ ]:
# 전체 페이지를 가져오기 전에 일부 페이지만 실행
page_list = []
# 데이터가 제대로 로드 되는지 앞부분 3페이지 정도만 확인하고 전체페이지를 가져오록 합니다.
# 처음부터 전체 페이지를 수집하면 중간에 오류가 나도 찾기가 어렵습니다.
# 일부만 우선 가져와 보고 잘 동작한다면 전체를 가져오도록 합니다.
all_page = 3
for page_no in trange(all_page + 1):
    one_page = get_seoul_covid20_100(page_no)
    one_page = pd.DataFrame(one_page["data"])
    page_list.append(one_page)
    # 서버에 한번에 너무 많은요청을 보내면 서버에 부담이 됩니다.
    # 서버에 부담을 주지 않기 위애 0.5초씩 쉬었다 가져옵니다.
    time.sleep(0.5)

100%|██████████| 4/4 [00:04<00:00,  1.07s/it]


In [ ]:
pd.concat(page_list)

,0,1,2,3,4,5,6
0,<p class='corona19_no'>12657</p>,43494,12.14.,동대문구,-,기타 확진자 접촉,<b class=''></b>
1,<p class='corona19_no'>12656</p>,43495,12.14.,관악구,-,기타 확진자 접촉,<b class=''></b>
2,<p class='corona19_no'>12655</p>,44350,12.14.,강서구,-,기타 확진자 접촉,<b class=''></b>
3,<p class='corona19_no'>12654</p>,44335,12.14.,광진구,-,동부구치소 관련,<b class=''></b>
4,<p class='corona19_no'>12653</p>,44334,12.14.,광진구,-,기타 확진자 접촉,<b class=''></b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>12362</p>,43293,12.13.,강서구,-,강서구 교회 관련,<b class='status2'></b>
96,<p class='corona19_no'>12361</p>,43294,12.13.,강서구,-,기타 확진자 접촉,<b class='status2'></b>
97,<p class='corona19_no'>12360</p>,43295,12.13.,강서구,-,기타 확진자 접촉,<b class='status2'></b>
98,<p class='corona19_no'>12359</p>,43304,12.13.,강서구,-,기타 확진자 접촉,<b class='status2'></b>


In [ ]:
def get_multi_page_list(start_page, end_page = 80):
    # 데이터가 제대로 로드 되는지 앞부분 3페이지 정도만 확인하고 전체페이지를 가져오록 합니다.
    
    page_list = []
    for page_no in trange(start_page, end_page + 1):
        one_page = get_seoul_covid20_100(page_no)
        if len(one_page["data"]) > 0:
            one_page = pd.DataFrame(one_page["data"])
            page_list.append(one_page)
            # 서버에 한번에 너무 많은요청을 보내면 서버에 부담이 됩니다.
            # 서버에 부담을 주지 않기 위애 0.5초씩 쉬었다 가져옵니다.
            time.sleep(0.5)
        else:
            # 수집된 값이 없다면 False를 반환합니다.
            # False 반환 시 수집한 리스트를 반환하도록 합니다.
            return page_list
    return page_list

In [ ]:
end_page = 4
# 시작페이지와 끝페이지를 꼭 확인해 주세요.
start_page = 1
# end_page = 88
page_list = get_multi_page_list(start_page, end_page)
# 데이터가 너무 많기 때문에 슬라이싱으로 1개만 미리보기
page_list[:1]

100%|██████████| 4/4 [00:04<00:00,  1.07s/it]


[                                   0      1  ...               5                 6
 0   <p class='corona19_no'>12657</p>  43494  ...       기타 확진자 접촉  <b class=''></b>
 1   <p class='corona19_no'>12656</p>  43495  ...       기타 확진자 접촉  <b class=''></b>
 2   <p class='corona19_no'>12655</p>  44350  ...       기타 확진자 접촉  <b class=''></b>
 3   <p class='corona19_no'>12654</p>  44335  ...        동부구치소 관련  <b class=''></b>
 4   <p class='corona19_no'>12653</p>  44334  ...       기타 확진자 접촉  <b class=''></b>
 ..                               ...    ...  ...             ...               ...
 95  <p class='corona19_no'>12562</p>  43955  ...       기타 확진자 접촉  <b class=''></b>
 96  <p class='corona19_no'>12561</p>  43980  ...  용산구 소재 건설현장 관련  <b class=''></b>
 97  <p class='corona19_no'>12560</p>  43686  ...        감염경로 조사중  <b class=''></b>
 98  <p class='corona19_no'>12559</p>  43693  ...       기타 확진자 접촉  <b class=''></b>
 99  <p class='corona19_no'>12558</p>  43972  ...        감염경로 조사중  <b class=

In [ ]:
# concat을 통해 하나의 데이터프레임으로 합쳐줍니다.
df_all = pd.concat(page_list)
df_all.shape

(400, 7)

In [ ]:
url = "http://www.seoul.go.kr/coronaV/coronaStatus.do"
url

table = pd.read_html(url)
df = table[3]
df.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황


In [ ]:
df_all.head(3)

,0,1,2,3,4,5,6
0,<p class='corona19_no'>12657</p>,43494,12.14.,동대문구,-,기타 확진자 접촉,<b class=''></b>
1,<p class='corona19_no'>12656</p>,43495,12.14.,관악구,-,기타 확진자 접촉,<b class=''></b>
2,<p class='corona19_no'>12655</p>,44350,12.14.,강서구,-,기타 확진자 접촉,<b class=''></b>


In [ ]:
cols = df.columns.tolist()
cols

['연번', '환자', '확진일', '거주지', '여행력', '접촉력', '퇴원현황']

In [ ]:
df_all.columns = cols
df_all.head(3)

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,<p class='corona19_no'>12657</p>,43494,12.14.,동대문구,-,기타 확진자 접촉,<b class=''></b>
1,<p class='corona19_no'>12656</p>,43495,12.14.,관악구,-,기타 확진자 접촉,<b class=''></b>
2,<p class='corona19_no'>12655</p>,44350,12.14.,강서구,-,기타 확진자 접촉,<b class=''></b>


In [ ]:
import re
def extract_number(num_string):
    if type(num_string) == str:
        num_string = num_string.replace("corona19", "")
        num = re.sub("[^0-9]", "", num_string)
        num = int(num)
        return num
    else:
        return num_string

In [ ]:
num_string = "<p class='corona19_no'>7625</p>"
extract_number(num_string)

7625

In [ ]:
df_all["연번"] = df_all["연번"].map(extract_number)

In [ ]:
df_all["연번"].head()

0    12657
1    12656
2    12655
3    12654
4    12653
Name: 연번, dtype: int64

In [ ]:
def extract_hangeul(origin_text):
    subtract_text = re.sub("[^가-힣]", "", origin_text)
    return subtract_text

In [ ]:
extract_hangeul("<b class='status1'>퇴원</b>")

'퇴원'

In [ ]:
df_all["퇴원현황"].value_counts()

<b class=''></b>             353
<b class='status2'></b>       46
<b class='status2'>사망</b>      1
Name: 퇴원현황, dtype: int64

In [ ]:
import numpy as np
df_all.loc[df_all["퇴원현황"].str.contains("퇴원"), "퇴원현황"] = "퇴원"
df_all.loc[df_all["퇴원현황"].str.contains("사망"), "퇴원현황"] = "사망"
df_all.loc[~df_all["퇴원현황"].str.contains("퇴원|사망"), "퇴원현황"] = np.nan
df_all["퇴원현황"].value_counts()

사망    1
Name: 퇴원현황, dtype: int64

In [ ]:
last_date = df_all.iloc[0]["확진일"]
last_date

'12.14.'

In [ ]:
date = last_date.replace(".", "_")
date

'12_14_'

In [ ]:
# 파일명을 만들어 줍니다.
# file_name

file_name = f"seoul-covid19-{date}.csv"
file_name

'seoul-covid19-12_14_.csv'

In [ ]:
# csv 파일로 저장합니다.
df_all.to_csv(file_name, index=False)

In [ ]:
# 제대로 저장되었는지 확인합니다.
pd.read_csv(file_name)

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,12657,43494,12.14.,동대문구,-,기타 확진자 접촉,NaN
1,12656,43495,12.14.,관악구,-,기타 확진자 접촉,NaN
2,12655,44350,12.14.,강서구,-,기타 확진자 접촉,NaN
3,12654,44335,12.14.,광진구,-,동부구치소 관련,NaN
4,12653,44334,12.14.,광진구,-,기타 확진자 접촉,NaN
...,...,...,...,...,...,...,...
395,12262,43166,12.13.,관악구,-,감염경로 조사중,NaN
396,12261,43063,12.13.,관악구,-,타시도 확진자 접촉,NaN
397,12260,43061,12.13.,관악구,-,감염경로 조사중,NaN
398,12259,43055,12.13.,광진구,-,기타 확진자 접촉,NaN
